# Speed Bumps by Neighborhood

### By: Jake Kressley

To start let's load in the neighborhood data.

In [1]:
import geopandas as gpd
import pandas as pd

neighborhoods_df = gpd.read_file('data/pittsburghpaneighborhoods-/Neighborhoods_.shp')
neighborhoods_df.head(3)

,OBJECTID,fid_blockg,statefp10,countyfp10,tractce10,blkgrpce10,geoid10,namelsad10,mtfcc10,funcstat10,...,created_us,created_da,last_edite,last_edi_1,temp,pghdb_sde_,GlobalID,Shape__Are,Shape__Len,geometry
0,1,233,42,003,140500,3,420031405003,Block Group 3,G5030,S,...,SDE,2020-08-14,SDE,2020-08-14,None,7968760.0,a642a6b6-a2a9-48d5-acf2-3e2ce87d4bc0,8.153647e+06,14047.065024,"POLYGON ((-79.90560 40.45522, -79.90553 40.455..."
1,2,203,42,003,140100,3,420031401003,Block Group 3,G5030,S,...,SDE,2020-08-14,SDE,2020-08-14,None,34078712.0,20d4605b-258e-441a-a57e-a28fdabb6d30,3.407739e+07,30347.886823,"POLYGON ((-79.91309 40.43804, -79.91321 40.438..."
2,3,86,42,003,101700,2,420031017002,Block Group 2,G5030,S,...,SDE,2020-08-14,SDE,2020-08-14,None,12751163.0,bf4065e7-c29c-48bd-b529-e7fd99077238,1.275116e+07,14577.441571,"POLYGON ((-79.93457 40.47217, -79.93455 40.472..."


To view the columns we want we can make a new dataframe that displays just those columns.

In [6]:
neighborhoods = pd.DataFrame(columns = ["Neighborhood", "Geometry"])
neighborhoods["Neighborhood"] = neighborhoods_df["hood"]
neighborhoods["Geometry"] = neighborhoods_df["geometry"]

neighborhoods.head(3)

,Neighborhood,Geometry
0,Point Breeze North,"POLYGON ((-79.90560 40.45522, -79.90553 40.455..."
1,Squirrel Hill North,"POLYGON ((-79.91309 40.43804, -79.91321 40.438..."
2,Garfield,"POLYGON ((-79.93457 40.47217, -79.93455 40.472..."
